## Loading Libraries

In [2]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
import glob
import os
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import  Dropout
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import chi2_contingency
import lightgbm as lgb
import networkx as nx
import warnings

In [4]:
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv(r"C:\Users\attafuro\Desktop\EV Charging Analysis\cleaned_traffic_data.csv")

In [6]:
df. head()

,Timestamp,Station,District,Route,Direction of Travel,Lane Type,Station Length,Samples,% Observed,Total Flow,...,Lane 5 Avg Speed,Lane 6 Flow,Lane 6 Avg Occ,Lane 6 Avg Speed,Lane 7 Flow,Lane 7 Avg Occ,Lane 7 Avg Speed,Lane 8 Flow,Lane 8 Avg Occ,Lane 8 Avg Speed
0,10/01/2024 00:00:00,308512,3,50,W,ML,3.995,197,0,497.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10/01/2024 00:00:00,311831,3,5,S,OR,NaN,101,92,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/01/2024 00:00:00,311832,3,5,S,FR,NaN,101,92,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10/01/2024 00:00:00,311844,3,5,N,OR,NaN,202,92,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10/01/2024 00:00:00,311847,3,5,N,OR,NaN,303,92,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.shape

(4114680, 42)

## Basic Exploration 

In [8]:
# Define the final selected columns
selected_columns = [
    "Timestamp", "Station", "Route", "Direction of Travel",
    "Total Flow", "Avg Speed", "% Observed","Samples","Lane Type"
]

# Keep only the selected columns
df = df[selected_columns]

In [9]:
df.head()

,Timestamp,Station,Route,Direction of Travel,Total Flow,Avg Speed,% Observed,Samples,Lane Type
0,10/01/2024 00:00:00,308512,50,W,497.0,64.1,0,197,ML
1,10/01/2024 00:00:00,311831,5,S,27.0,NaN,92,101,OR
2,10/01/2024 00:00:00,311832,5,S,78.0,NaN,92,101,FR
3,10/01/2024 00:00:00,311844,5,N,43.0,NaN,92,202,OR
4,10/01/2024 00:00:00,311847,5,N,73.0,NaN,92,303,OR


In [10]:
df.isna().sum()

Timestamp                    0
Station                      0
Route                        0
Direction of Travel          0
Total Flow              303776
Avg Speed              1582594
% Observed                   0
Samples                      0
Lane Type                    0
dtype: int64

In [11]:
df.drop(columns=["Avg Speed"], inplace=True)

In [12]:
df.head()

,Timestamp,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type
0,10/01/2024 00:00:00,308512,50,W,497.0,0,197,ML
1,10/01/2024 00:00:00,311831,5,S,27.0,92,101,OR
2,10/01/2024 00:00:00,311832,5,S,78.0,92,101,FR
3,10/01/2024 00:00:00,311844,5,N,43.0,92,202,OR
4,10/01/2024 00:00:00,311847,5,N,73.0,92,303,OR


In [13]:
df.isna().sum()

Timestamp                   0
Station                     0
Route                       0
Direction of Travel         0
Total Flow             303776
% Observed                  0
Samples                     0
Lane Type                   0
dtype: int64

In [14]:
df = df[df['Total Flow'].notna()]

In [15]:
df.isna().sum()

Timestamp              0
Station                0
Route                  0
Direction of Travel    0
Total Flow             0
% Observed             0
Samples                0
Lane Type              0
dtype: int64

In [16]:
df.head()

,Timestamp,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type
0,10/01/2024 00:00:00,308512,50,W,497.0,0,197,ML
1,10/01/2024 00:00:00,311831,5,S,27.0,92,101,OR
2,10/01/2024 00:00:00,311832,5,S,78.0,92,101,FR
3,10/01/2024 00:00:00,311844,5,N,43.0,92,202,OR
4,10/01/2024 00:00:00,311847,5,N,73.0,92,303,OR


In [17]:
df.dtypes

Timestamp               object
Station                  int64
Route                    int64
Direction of Travel     object
Total Flow             float64
% Observed               int64
Samples                  int64
Lane Type               object
dtype: object

In [18]:
df.duplicated().sum()

0

In [19]:
df.reset_index().duplicated(subset=['Timestamp', 'Station']).sum()

0

In [20]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)
df.sort_index(inplace=True)

In [21]:
# Convert Station to categorical type
df['Station'] = df['Station'].astype('category')

In [22]:
df.head()

,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type
Timestamp,,,,,,,
2024-10-01,308512,50,W,497.0,0,197,ML
2024-10-01,311831,5,S,27.0,92,101,OR
2024-10-01,311832,5,S,78.0,92,101,FR
2024-10-01,311844,5,N,43.0,92,202,OR
2024-10-01,311847,5,N,73.0,92,303,OR


## Feature Engineering

In [23]:
df['Hour'] = df.index.hour
df['Day'] = df.index.day
df['Weekday'] = df.index.weekday
df['IsWeekend'] = df['Weekday'].isin([5, 6]).astype(int)

In [24]:
for lag in range(1, 73):  # Lags 1 to 72
    df[f'TotalFlow_lag_{lag}'] = df['Total Flow'].shift(lag)

# Drop the rows that now have NaNs in these lag features (first 72 rows)
df.dropna(inplace=True)

In [25]:
df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
df['Weekday_sin'] = np.sin(2 * np.pi * df['Weekday'] / 7)
df['Weekday_cos'] = np.cos(2 * np.pi * df['Weekday'] / 7)

In [26]:
df['Flow_roll_mean_6h'] = df['Total Flow'].rolling(window=6).mean()
df['Flow_roll_std_6h'] = df['Total Flow'].rolling(window=6).std()
df['Flow_roll_max_6h'] = df['Total Flow'].rolling(window=6).max()
df['Flow_roll_min_6h'] = df['Total Flow'].rolling(window=6).min()

In [27]:
for window in [3, 12, 24, 48, 72]:
    df[f'Flow_mean_{window}'] = df['Total Flow'].rolling(window=window).mean()
    df[f'Flow_std_{window}'] = df['Total Flow'].rolling(window=window).std()
    df[f'Flow_min_{window}'] = df['Total Flow'].rolling(window=window).min()
    df[f'Flow_max_{window}'] = df['Total Flow'].rolling(window=window).max()

In [28]:
df.dropna(inplace=True)

In [29]:
custom_holidays = [
    pd.to_datetime("2024-12-25"),
    pd.to_datetime("2024-12-31"),
    pd.to_datetime("2024-11-24"),
]
df['IsHoliday'] = df.index.to_series().apply(lambda x: 1 if x in custom_holidays else 0)

In [30]:
df.head()

,Station,Route,Direction of Travel,Total Flow,% Observed,Samples,Lane Type,Hour,Day,Weekday,...,Flow_max_24,Flow_mean_48,Flow_std_48,Flow_min_48,Flow_max_48,Flow_mean_72,Flow_std_72,Flow_min_72,Flow_max_72,IsHoliday
Timestamp,,,,,,,,,,,,,,,,,,,,,
2024-10-01,313344,50,W,190.0,92,202,ML,0,1,1,...,1068.0,395.833333,570.838627,3.0,3419.0,435.736111,522.596712,3.0,3419.0,0
2024-10-01,313349,50,W,190.0,92,202,ML,0,1,1,...,1068.0,399.458333,568.924516,3.0,3419.0,437.486111,521.544267,3.0,3419.0,0
2024-10-01,313368,80,W,787.0,100,719,ML,0,1,1,...,1068.0,404.375000,571.276906,3.0,3419.0,440.069444,522.824677,3.0,3419.0,0
2024-10-01,313386,80,W,1050.0,92,404,ML,0,1,1,...,1068.0,426.000000,575.736716,3.0,3419.0,443.555556,526.078366,3.0,3419.0,0
2024-10-01,313393,80,E,1064.0,92,505,ML,0,1,1,...,1068.0,376.937500,383.507686,3.0,1525.0,457.500000,529.066225,3.0,3419.0,0


In [31]:
for window in [12, 24, 48, 72]:
    df[f'Flow_CV_{window}'] = df[f'Flow_std_{window}'] / df[f'Flow_mean_{window}']

In [32]:
df.dropna(inplace=True)

In [33]:
# Group by station and calculate mean and std of Total Flow
station_stats = df.groupby('Station')['Total Flow'].agg(['mean', 'std'])
station_stats['CV'] = station_stats['std'] / station_stats['mean']
station_stats.dropna(inplace=True)  

In [34]:
df['IsHoliday'] = df['IsHoliday'].astype('category')
df['Route'] = df['Route'].astype('category')

In [35]:
#  Keep only numeric features
numeric_df = df.select_dtypes(include='number')

In [36]:
cat_features = [
    'Station', 'Route', 'Direction of Travel', 'Lane Type',
    'Weekday', 'IsWeekend', 'IsHoliday'
]

for col in cat_features:
    df[col] = df[col].astype('category')

In [37]:
# Define target and features
target = 'Total Flow'
features = [col for col in df.columns if col != target]

In [38]:
# Final feature groups
flow_features = [
    'Flow_mean_3', 'Flow_std_3', 'Flow_max_3', 'Flow_min_3',
    'Flow_mean_72', 'Flow_std_72', 'Flow_std_48', 'Flow_max_24',
    'Flow_mean_24', 'Flow_mean_48',
    'Flow_roll_std_6h', 'Flow_roll_max_6h',
    'Flow_CV_48'
]

lag_features = [
    'TotalFlow_lag_1', 'TotalFlow_lag_2', 'TotalFlow_lag_3',
    'TotalFlow_lag_7', 'TotalFlow_lag_8',
    'TotalFlow_lag_15', 'TotalFlow_lag_18'
]

time_features = [
    'Hour', 'Hour_cos', 'Hour_sin', 'Weekday_cos'
]

quality_features = [
    'Samples', '% Observed'
]

# Copy original DataFrame
df_model = df.copy()

# One-hot encode low-cardinality features
df_model = pd.get_dummies(df_model, columns=['IsHoliday', 'Direction of Travel'], drop_first=True)

# Label encode high-cardinality categorical features
for col in ['Station', 'Route', 'Lane Type']:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])

# Capture dummy columns dynamically
dummy_columns = [col for col in df_model.columns if col.startswith('IsHoliday_') or col.startswith('Direction of Travel_')]

# Final feature list
categorical_features = ['Lane Type', 'Route', 'Station']
selected_features = flow_features + lag_features + time_features + quality_features + categorical_features + dummy_columns


In [40]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.spatial.distance import pdist, squareform
import torch
from sklearn.preprocessing import LabelEncoder

#  Build CV Graph from Flow_CV_48 using ORIGINAL station IDs
cv_df = df.groupby('Station')['Flow_CV_48'].mean().reset_index()
stations = cv_df['Station'].values
cv_values = cv_df['Flow_CV_48'].values

#  Compute pairwise cityblock distances
dist_matrix = squareform(pdist(cv_values[:, None], metric='cityblock'))

#  Build NetworkX graph G with threshold
threshold = 0.5
G = nx.Graph()
G.add_nodes_from(stations)

for i in range(len(stations)):
    for j in range(i + 1, len(stations)):
        if dist_matrix[i, j] <= threshold:
            G.add_edge(stations[i], stations[j])

print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

#  Build neighbor lookup (optional)
neighbor_map = {station: list(G.neighbors(station)) for station in G.nodes}

#  Build normalized adjacency matrix A_hat using ORIGINAL station IDs
def build_adjacency_matrix(G, station_ids):
    num_nodes = len(station_ids)
    node_to_idx = {node: idx for idx, node in enumerate(station_ids)}
    A = np.zeros((num_nodes, num_nodes))

    for src, dst in G.edges():
        i, j = node_to_idx[src], node_to_idx[dst]
        A[i, j] = 1
        A[j, i] = 1  # Undirected

    A += np.eye(num_nodes)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(A.sum(axis=1)))
    A_hat = D_inv_sqrt @ A @ D_inv_sqrt

    return torch.tensor(A_hat, dtype=torch.float32)

# Use original station IDs from df (not encoded!)
station_ids = sorted(df['Station'].unique())
A_hat = build_adjacency_matrix(G, station_ids)

print("Normalized adjacency matrix shape:", A_hat.shape)

le = LabelEncoder()
df_model['Station_encoded'] = le.fit_transform(df_model['Station'])

# Store mappings for future reference
station_id_to_label = dict(zip(le.classes_, le.transform(le.classes_)))
label_to_station_id = dict(zip(le.transform(le.classes_), le.classes_))


Graph has 1806 nodes and 1295394 edges
Normalized adjacency matrix shape: torch.Size([1806, 1806])


In [46]:
import torch.nn as nn
import torch

class GraphConvolution(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphConvolution, self).__init__()
        self.theta = nn.Parameter(torch.FloatTensor(in_channels, out_channels))
        nn.init.xavier_uniform_(self.theta)

    def forward(self, X, A_hat):
        # X: (batch, nodes, in_channels)
        out = torch.einsum("bnc,cd->bnd", X, self.theta)  # (batch, nodes, out_channels)
        out = torch.einsum("ij,bjd->bid", A_hat, out)     # apply spatial filter
        return out

In [47]:
class STBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dropout=0.3):
        super(STBlock, self).__init__()
        self.temp_conv = nn.Conv2d(in_channels, out_channels, kernel_size=(kernel_size, 1), padding=(1, 0))
        self.graph_conv = GraphConvolution(out_channels, out_channels)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, X, A_hat):
        # X: (batch, seq_len, num_nodes, in_channels)
        X = X.permute(0, 3, 1, 2)  # → (batch, in_channels, seq_len, num_nodes)
        out = self.temp_conv(X)   # → (batch, out_channels, seq_len, num_nodes)
        out = out.permute(0, 2, 3, 1)  # → (batch, seq_len, num_nodes, out_channels)

        out_gc = []
        for t in range(out.size(1)):
            out_t = self.graph_conv(out[:, t], A_hat)  # → (batch, num_nodes, out_channels)
            out_gc.append(out_t)
        out_gc = torch.stack(out_gc, dim=1)  # → (batch, seq_len, num_nodes, out_channels)

        out_gc = out_gc.permute(0, 3, 1, 2)  # → (batch, out_channels, seq_len, num_nodes)
        out_gc = self.bn(out_gc)
        out_gc = self.relu(out_gc)
        out_gc = self.dropout(out_gc)
        return out_gc.permute(0, 2, 3, 1)  # → (batch, seq_len, num_nodes, out_channels)


In [50]:
from collections import defaultdict
import numpy as np

def create_sequences(df, selected_features, station_col, target_col, seq_len, horizon):
    stations = sorted(df[station_col].unique())
    station_data = defaultdict(list)
    station_targets = defaultdict(list)

    #  Extract valid sequences
    for station in stations:
        df_s = df[df[station_col] == station].sort_index()
        X = df_s[selected_features].values
        y = df_s[target_col].values

        for i in range(len(df_s) - seq_len - horizon + 1):
            X_seq = X[i:i+seq_len]
            y_val = y[i+seq_len+horizon-1]
            station_data[station].append(X_seq)
            station_targets[station].append(y_val)

    # Drop stations with no valid sequences
    valid_stations = [s for s in stations if len(station_data[s]) > 0]
    min_len = min(len(station_data[s]) for s in valid_stations)
    print("Using minimum aligned sequence length:", min_len)

    # Step 3: Align all stations to same number of samples
    X_final, Y_final = [], []

    for t in range(min_len):
        snapshot, target_snapshot = [], []
        for s in valid_stations:
            snapshot.append(station_data[s][t])
            target_snapshot.append(station_targets[s][t])
        X_final.append(np.stack(snapshot, axis=0))      # (num_nodes, seq_len, features)
        Y_final.append(np.array(target_snapshot))       # (num_nodes,)

    X_final = np.stack(X_final)                          # (N, num_nodes, seq_len, features)
    X_final = np.transpose(X_final, (0, 2, 1, 3))        # (N, seq_len, num_nodes, features)
    Y_final = np.stack(Y_final)                          # (N, num_nodes)

    return X_final.astype(np.float32), Y_final.astype(np.float32)


In [53]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# Define corrected STGCN model
class STGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, A_hat):
        super(STGCN, self).__init__()
        self.A_hat = A_hat
        self.gc1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=(1, 1))
        self.relu = nn.ReLU()
        self.final_proj = nn.Linear(hidden_channels, 1)

    def forward(self, X, A_hat):
        # X: (batch, seq_len, num_nodes, features)
        X = X.permute(0, 3, 1, 2)  # (batch, features, seq_len, num_nodes)
        X = self.gc1(X)            # (batch, hidden_channels, seq_len, num_nodes)
        X = self.relu(X)
        X = X.mean(dim=2)          # temporal average pooling → (batch, hidden_channels, num_nodes)
        X = X.permute(0, 2, 1)     # (batch, num_nodes, hidden_channels)
        out = self.final_proj(X).squeeze(-1)  # (batch, num_nodes)
        return out

# Hyperparameters
seq_len = 12
batch_size = 32
epochs = 20
learning_rate = 0.001
hidden_channels = 32

horizon_map = {1: '12h', 2: '24h', 4: '48h', 6: '72h'}
results = {}

for horizon, label in horizon_map.items():
    print(f"\n===== Forecast Horizon: {label} (horizon={horizon}) =====")

    # Create sequences
    X, Y = create_sequences(
        df_model,
        selected_features,
        station_col='Station',
        target_col='Total Flow',
        seq_len=seq_len,
        horizon=horizon
    )

    # Train/Val Split
    split = int(0.8 * len(X))
    X_train, X_val = X[:split], X[split:]
    Y_train, Y_val = Y[:split], Y[split:]

    # Convert to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    Y_train = torch.tensor(Y_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    Y_val = torch.tensor(Y_val, dtype=torch.float32)

    train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, Y_val), batch_size=batch_size, shuffle=False)

    in_channels = X.shape[-1]
    model = STGCN(in_channels=in_channels, hidden_channels=hidden_channels, A_hat=A_hat)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.L1Loss()  # MAE

    # Training
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch_X, batch_Y in train_loader:
            optimizer.zero_grad()
            out = model(batch_X, A_hat)
            loss = loss_fn(out, batch_Y)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {np.mean(train_losses):.4f}")

    # Evaluation
    model.eval()
    Y_preds, Y_true = [], []
    with torch.no_grad():
        for batch_X, batch_Y in val_loader:
            out = model(batch_X, A_hat)
            Y_preds.append(out.numpy())
            Y_true.append(batch_Y.numpy())

    Y_preds = np.concatenate(Y_preds, axis=0)
    Y_true = np.concatenate(Y_true, axis=0)

    mae = mean_absolute_error(Y_true.flatten(), Y_preds.flatten())
    rmse = mean_squared_error(Y_true.flatten(), Y_preds.flatten(), squared=False)

    print(f"MAE ({label}): {mae:.2f}")
    print(f"RMSE ({label}): {rmse:.2f}")
    results[label] = {'MAE': mae, 'RMSE': rmse}




===== Forecast Horizon: 12h (horizon=1) =====
Using minimum aligned sequence length: 12
Epoch 1/20 - Train Loss: 1187.9296
Epoch 2/20 - Train Loss: 1162.8612
Epoch 3/20 - Train Loss: 1140.1461
Epoch 4/20 - Train Loss: 1119.8931
Epoch 5/20 - Train Loss: 1102.0348
Epoch 6/20 - Train Loss: 1086.5231
Epoch 7/20 - Train Loss: 1073.1499
Epoch 8/20 - Train Loss: 1061.6300
Epoch 9/20 - Train Loss: 1051.7760
Epoch 10/20 - Train Loss: 1043.4690
Epoch 11/20 - Train Loss: 1036.6628
Epoch 12/20 - Train Loss: 1031.1604
Epoch 13/20 - Train Loss: 1026.8378
Epoch 14/20 - Train Loss: 1023.6129
Epoch 15/20 - Train Loss: 1021.3451
Epoch 16/20 - Train Loss: 1019.8205
Epoch 17/20 - Train Loss: 1018.9062
Epoch 18/20 - Train Loss: 1018.3884
Epoch 19/20 - Train Loss: 1018.0207
Epoch 20/20 - Train Loss: 1017.6587
MAE (12h): 749.46
RMSE (12h): 892.39

===== Forecast Horizon: 24h (horizon=2) =====
Using minimum aligned sequence length: 11
Epoch 1/20 - Train Loss: 2001.7181
Epoch 2/20 - Train Loss: 1959.6017
Epoc

,MAE,RMSE
Forecast Horizon,,
12h,749.456909,892.392273
24h,388.228912,666.230530
48h,593.774963,697.883240
72h,688.708679,839.535217


In [55]:
# Display final results
results_df = pd.DataFrame(results).T
results_df.index.name = 'Forecast Horizon'
display(results_df)


,MAE,RMSE
Forecast Horizon,,
12h,749.456909,892.392273
24h,388.228912,666.230530
48h,593.774963,697.883240
72h,688.708679,839.535217
